# **Exploratory Data Analysis of Ecologic Variables**

[to be written]

In [ ]:
# Libraries
import pandas as pd
from scipy import stats
import scikit_posthocs as sp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data
df = pd.read_csv('AVONET.csv' )

## **1. Body Size (Mass)**

### **1.1. Mass and Habitat**

To choose the proper statistical test, distribution of Mass is explored based on habitat. Shapiro-Wilk test is done, in which

* H0: The Mass is normally distributed within Habitat groups.
* H1: The Mass is not normally distributed within Habitat groups.

In [ ]:
HABITAT_LEVELS = ['Closed', 'Open', 'Aquatic']

df_habitat_mass = df[['Habitat', 'Mass']].copy()
df_habitat_mass = df_habitat_mass[df_habitat_mass['Habitat'].isin(HABITAT_LEVELS)]

###

for group_name in sorted(HABITAT_LEVELS):
    group_data = df_habitat_mass[df_habitat_mass['Habitat'] == group_name]['Mass']

    shapiro_w, shapiro_p = stats.shapiro(group_data)

    print(f"[{group_name}] N={len(group_data)} | p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as ANOVA cannot be used. As an alternative, Kruskal-Wallis test is used, in which

* H0: There is no statistically significant difference in the distribution of Mass across Habitat groups.
* H1: The distribution of Mass differs among at least two Habitat groups.

In [ ]:
groups = sorted(df_habitat_mass['Habitat'].unique())
data_groups = [df_habitat_mass[df_habitat_mass['Habitat'] == g]['Mass'] for g in groups]

h_stat, p_val = stats.kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of Mass among at least two Habitat groups. To see which Habitat groups differ in terms of Mass, Dunn's test is done, in which

* H0: The distribution of Mass is the same for the two Habitat groups tested.
* H1: The distribution of Mass differs between the two Habitat groups tested.

In [ ]:
dunn_results = sp.posthoc_dunn(a=df_habitat_mass, 
                                val_col='Mass', 
                                group_col='Habitat', 
                                p_adjust='bonferroni')

groups = sorted(df_habitat_mass['Habitat'].unique())
dunn_results.columns = groups
dunn_results.index = groups

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in Mass distribution between every single pair of Habitat groups. Finally, to illustrate the distribution, a boxplot is created (with log(Mass) for better visualization).

In [ ]:
df_habitat_mass['log_Mass'] = np.log10(df_habitat_mass['Mass'])
df_plot = df_habitat_mass[['Habitat', 'log_Mass']].copy()
df_plot = df_plot[df_plot['Habitat'].isin(HABITAT_LEVELS)]
df['Habitat'] = df['Habitat'].astype(str)

plt.figure(figsize=(10, 6))
sns.boxplot(x='Habitat', y='log_Mass', data=df_plot) 
plt.title('Log(Mass) by Habitat')
plt.xlabel('Habitat')
plt.ylabel('Log(Mass) (log(g))') 
plt.tight_layout()

plt.show()

### **1.2. Mass and Migration**

To choose the proper statistical test, distribution of Mass is explored based on migration behaviour. Shapiro-Wilk test is done, in which

* H0: The mass is normally distributed within Migratory classes.
* H1: The mass is not normally distributed within Migratory classes.

In [ ]:
MIGRATION_LEVELS = ['Sedentary', 'Partial', 'Migratory']

df_migration_mass = df[['Migration', 'Mass']].copy()
df_migration_mass = df_migration_mass[df_migration_mass['Migration'].isin(MIGRATION_LEVELS)]

###

for group_name in sorted(MIGRATION_LEVELS):
    group_data = df_migration_mass[df_migration_mass['Migration'] == group_name]['Mass']

    shapiro_w, shapiro_p = stats.shapiro(group_data)

    print(f"[{group_name}] N={len(group_data)} | p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as ANOVA cannot be used. As an alternative, Kruskal-Wallis test is used, in which

* H0: There is no statistically significant difference in the distribution of Mass across Migratory classes.
* H1: The distribution of Mass differs among at least two Migratory classes.

In [ ]:
groups = sorted(df_migration_mass['Migration'].unique())
data_groups = [df_migration_mass[df_migration_mass['Migration'] == g]['Mass'] for g in groups]

h_stat, p_val = stats.kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of Mass among at least two Migratory classes. To see which Migratory classes differ in terms of Mass, Dunn's test is done, in which

* H0: The distribution of Mass is the same for the two Migratory classes tested.
* H1: The distribution of Mass differs between the two Migratory classes tested.

In [ ]:
dunn_results = sp.posthoc_dunn(a=df_migration_mass, 
                                val_col='Mass', 
                                group_col='Migration', 
                                p_adjust='bonferroni')

groups = sorted(df_migration_mass['Migration'].unique())
dunn_results.columns = groups
dunn_results.index = groups

print(dunn_results)

Between Sedentary and Migratory classes, p > 0.05, thus the null hypothesis (H0) is failed to reject. However, between Partial-Sedentary and Partial-Migratory, p << 0.05, thus there are statistically significant differences in Mass distribution between these Migratory classes. Finally, to illustrate the distribution, a boxplot is created (with log(Mass) for better visualization).

In [ ]:
df_migration_mass['log_Mass'] = np.log10(df_migration_mass['Mass'])
df_plot = df_migration_mass[['Migration', 'log_Mass']].copy()
df_plot = df_plot[df_plot['Migration'].isin(MIGRATION_LEVELS)]
df['Migration'] = df['Migration'].astype(str)

plt.figure(figsize=(10, 6))
sns.boxplot(x='Migration', y='log_Mass', data=df_plot) 
plt.title('Log(Mass) by Migration')
plt.xlabel('Migration')
plt.ylabel('Log(Mass) (log(g))') 
plt.tight_layout()

plt.show()

### **1.3. Mass and Nutrition**

To choose the proper statistical test, distribution of Mass is explored based on nutrition. Shapiro-Wilk test is done, in which

* H0: The mass is normally distributed within Trophic levels.
* H1: The mass is not normally distributed within Trophic levels.

In [ ]:
TROPHIC_LEVELS = ['Carnivore', 'Herbivore', 'Omnivore']

df_trophic_mass = df[['Trophic.Level', 'Mass']].copy()
df_trophic_mass = df_trophic_mass[df_trophic_mass['Trophic.Level'].isin(TROPHIC_LEVELS)]

###

for group_name in sorted(TROPHIC_LEVELS):
    group_data = df_trophic_mass[df_trophic_mass['Trophic.Level'] == group_name]['Mass']

    shapiro_w, shapiro_p = stats.shapiro(group_data)

    print(f"[{group_name}] N={len(group_data)} | p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as ANOVA cannot be used. As an alternative, Kruskal-Wallis test is used, in which

* H0: There is no statistically significant difference in the distribution of Mass across Trophic levels.
* H1: The distribution of Mass differs among at least two Trophic levels.

In [ ]:
groups = sorted(df_trophic_mass['Trophic.Level'].unique())
data_groups = [df_trophic_mass[df_trophic_mass['Trophic.Level'] == g]['Mass'] for g in groups]

h_stat, p_val = stats.kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of Mass among at least two Trophic levels. To see which Trophic levels differ in terms of Mass, Dunn's test is done, in which

* H0: The distribution of Mass is the same for the two Trophic levels tested.
* H1: The distribution of Mass differs between the two Trophic levels tested.

In [ ]:
dunn_results = sp.posthoc_dunn(a=df_trophic_mass, 
                                val_col='Mass', 
                                group_col='Trophic.Level', 
                                p_adjust='bonferroni')

groups = sorted(df_trophic_mass['Trophic.Level'].unique())
dunn_results.columns = groups
dunn_results.index = groups

print(dunn_results)

Since p < 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in Mass distribution between every single pair of Trophic levels. Finally, to illustrate the distribution, a boxplot is created (with log(Mass) for better visualization).

In [ ]:
df_trophic_mass['log_Mass'] = np.log10(df_trophic_mass['Mass'])
df_plot = df_trophic_mass[['Trophic.Level', 'log_Mass']].copy()
df_plot = df_plot[df_plot['Trophic.Level'].isin(TROPHIC_LEVELS)]
df['Trophic.Level'] = df['Trophic.Level'].astype(str)

plt.figure(figsize=(10, 6))
sns.boxplot(x='Trophic.Level', y='log_Mass', data=df_plot) 
plt.title('Log(Mass) by Trophic.Level')
plt.xlabel('Trophic.Level')
plt.ylabel('Log(Mass) (log(g))') 
plt.tight_layout()

plt.show()

## **2. Leg Size (Tarsus Length)**

### **2.1. Tarsus Length and Nutrition**

To choose the proper statistical test, distribution of Tarsus Length is explored based on nutrition. Shapiro-Wilk test is done, in which

* H0: The tarsus length is normally distributed within Trophic levels.
* H1: The tarsus length is not normally distributed within Trophic levels.

In [ ]:
df_trophic_tarsus = df[['Trophic.Level', 'Tarsus.Length']].copy()
df_trophic_tarsus = df_trophic_tarsus[df_trophic_tarsus['Trophic.Level'].isin(TROPHIC_LEVELS)]

###

for group_name in sorted(TROPHIC_LEVELS):
    group_data = df_trophic_tarsus[df_trophic_tarsus['Trophic.Level'] == group_name]['Tarsus.Length']

    shapiro_w, shapiro_p = stats.shapiro(group_data)

    print(f"[{group_name}] N={len(group_data)} | p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as ANOVA cannot be used. As an alternative, Kruskal-Wallis test is used, in which

* H0: There is no statistically significant difference in the distribution of Tarsus Length across Trophic levels.
* H1: The distribution of Tarsus Length differs among at least two Trophic levels.

In [ ]:
groups = sorted(df_trophic_tarsus['Trophic.Level'].unique())
data_groups = [df_trophic_tarsus[df_trophic_tarsus['Trophic.Level'] == g]['Tarsus.Length'] for g in groups]

h_stat, p_val = stats.kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of Tarsus Length among at least two Trophic levels. To see which Trophic levels differ in terms of Tarsus Length, Dunn's test is done, in which

* H0: The distribution of Tarsus Length is the same for the two Trophic levels tested.
* H1: The distribution of Tarsus Length differs between the two Trophic levels tested.

In [ ]:
dunn_results = sp.posthoc_dunn(a=df_trophic_tarsus, 
                                val_col='Tarsus.Length', 
                                group_col='Trophic.Level', 
                                p_adjust='bonferroni')

groups = sorted(df_trophic_tarsus['Trophic.Level'].unique())
dunn_results.columns = groups
dunn_results.index = groups

print(dunn_results)

Since p < 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in Tarsus Length distribution between every single pair of Trophic levels. Finally, to illustrate the distribution, a boxplot is created (with log(Length) for better visualization).

In [ ]:
df_trophic_tarsus['log_TarsusLength'] = np.log10(df_trophic_tarsus['Tarsus.Length'])
df_plot = df_trophic_tarsus[['Trophic.Level', 'log_TarsusLength']].copy()
df_plot = df_plot[df_plot['Trophic.Level'].isin(TROPHIC_LEVELS)]
df['Trophic.Level'] = df['Trophic.Level'].astype(str)

plt.figure(figsize=(10, 6))
sns.boxplot(x='Trophic.Level', y='log_TarsusLength', data=df_plot) 
plt.title('Log(Tarsus Length) by Trophic Level')
plt.xlabel('Trophic Level')
plt.ylabel('Log(Length) (log(mm))') 
plt.tight_layout()

plt.show()

### **2.2. Tarsus Length and Habitat**

To choose the proper statistical test, distribution of Tarsus length is explored based on habitat. Shapiro-Wilk test is done, in which

* H0: The Tarsus length is normally distributed within Habitat groups.
* H1: The Tarsus length is not normally distributed within Habitat groups.

In [ ]:
df_habitat_tarsus = df[['Habitat', 'Tarsus.Length']].copy()
df_habitat_tarsus = df_habitat_tarsus[df_habitat_tarsus['Habitat'].isin(HABITAT_LEVELS)]

###

for group_name in sorted(HABITAT_LEVELS):
    group_data = df_habitat_tarsus[df_habitat_tarsus['Habitat'] == group_name]['Tarsus.Length']

    shapiro_w, shapiro_p = stats.shapiro(group_data)

    print(f"[{group_name}] N={len(group_data)} | p-value: {shapiro_p:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. The data is not normally distributed. Therefore, parametric tests such as ANOVA cannot be used. As an alternative, Kruskal-Wallis test is used, in which

* H0: There is no statistically significant difference in the distribution of Tarsus length across Habitat groups.
* H1: The distribution of Tarsus length differs among at least two Habitat groups.

In [ ]:
groups = sorted(df_habitat_tarsus['Habitat'].unique())
data_groups = [df_habitat_tarsus[df_habitat_tarsus['Habitat'] == g]['Tarsus.Length'] for g in groups]

h_stat, p_val = stats.kruskal(*data_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {p_val:.4e}")

Since p << 0.05, the null hypothesis (H0) is rejected. There is a statistically significant difference in the distribution of Tarsus length among at least two Habitat groups. To see which Habitat groups differ in terms of Tarsus length, Dunn's test is done, in which

* H0: The distribution of Tarsus length is the same for the two Habitat groups tested.
* H1: The distribution of Tarsus length differs between the two Habitat groups tested.

In [ ]:
dunn_results = sp.posthoc_dunn(a=df_habitat_tarsus, 
                                val_col='Tarsus.Length', 
                                group_col='Habitat', 
                                p_adjust='bonferroni')

groups = sorted(df_habitat_tarsus['Habitat'].unique())
dunn_results.columns = groups
dunn_results.index = groups

print(dunn_results)

Since p << 0.05, the null hypothesis (H0) is rejected; there are statistically significant differences in Tarsus length distribution between every single pair of Habitat groups. Finally, to illustrate the distribution, a boxplot is created (with log(Length) for better visualization).

In [ ]:
df_habitat_tarsus['log_TarsusLength'] = np.log10(df_habitat_tarsus['Tarsus.Length'])
df_plot = df_habitat_tarsus[['Habitat', 'log_TarsusLength']].copy()
df_plot = df_plot[df_plot['Habitat'].isin(HABITAT_LEVELS)]
df['Habitat'] = df['Habitat'].astype(str)

plt.figure(figsize=(10, 6))
sns.boxplot(x='Habitat', y='log_TarsusLength', data=df_plot) 
plt.title('Log(Tarsus Length) by Habitat')
plt.xlabel('Habitat')
plt.ylabel('Log(Length) (log(mm))') 
plt.tight_layout()

plt.show()